In [ ]:
import pyspiel
import numpy as np
import gym, gym.spaces
import math
import random
import numpy as np
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [ ]:
GAME = pyspiel.load_game('bridge(use_double_dummy_result=true)')

class BridgeEnv(gym.Env):
    """Custom Environment that follows gym interface"""

    def __init__(self):
        super(BridgeEnv, self).__init__()    # Define action and observation space
        self.action_space = gym.spaces.Box(low=0, high=1, shape=(38,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(571,), dtype=np.float32)
        self.reset()

    def reset(self):
        self.state = self.generate_random_game()
        return np.array(self.state.observation_tensor())

    def step(self, action):
        action = self.pick_action(action)

        self.state.apply_action(action+52)
        
        if self.state.current_phase() == 3:
            return self.calculate_terminal_reward(action)
        
        # random opposing team
        self.state.apply_action(random.choice(self.state.legal_actions()))

        if self.state.current_phase() == 3:
            return self.calculate_terminal_reward(action, invert=True)

        return self.calculate_default_reward(action)

    def calculate_default_reward(self, action):
        obs = np.array(self.state.observation_tensor())
        reward = 0
        done = False
        return obs, reward, done, {"action": action}

    def calculate_terminal_reward(self, action, invert=False):
        obs = np.zeros(571)
        reward = self.state.score_by_contract()[self.state.contract_index()]
        if invert:
            reward = -reward
        done = True
        return obs, reward, done, {"action": action}

    def pick_action(self, action_vector):
        action_vector = self.softmax(action_vector)
        legal_action_mask = np.array(self.state.legal_actions_mask())[52:52+self.action_space.shape[0]]
        masked_action_vector = action_vector*legal_action_mask
        action = np.argmax(masked_action_vector)

        return action


    def softmax(self, x):
        y = np.exp(x - np.max(x))
        f_x = y / np.sum(y)
        return f_x

    def generate_random_game(self): 
        state = GAME.new_initial_state()
        # deal all 52 cards randomly
        for i in np.random.choice(52, size=(52,), replace=False):
            state.apply_action(i)
        return state
    

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN(nn.Module):

    def __init__(self, inputs, outputs):
        super(DQN, self).__init__()
        self.num_layers = 3
        inputs = [inputs] + [1024] * self.num_layers
        outputs =           [1024] * self.num_layers + [outputs]

        self.layers = []
        for index, (input, output) in enumerate(zip(inputs,outputs)):
            self.layers.append(nn.Linear(input,output))
            self.add_module(f"Linear{index}", self.layers[-1])
            self.layers.append(nn.ReLU())
            self.add_module(f"ReLu{index}", self.layers[-1])

            if index < self.num_layers - 1:
                self.layers.append(nn.BatchNorm1d(output))
                self.add_module(f"BatchNorm{index}", self.layers[-1])

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
def select_action(state, env):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            policy_net.eval()
            return policy_net(state).cpu()
    else:
        choice = random.choice(env.state.legal_actions())-52
        return torch.tensor(env.action_space.sample().reshape((1,-1)), device=device, dtype=torch.long)

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
                                                
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch.type(torch.int64).to(device).unsqueeze(1)).squeeze()

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    target_net.train()
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

    return loss

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = BridgeEnv()

# Get number of actions from gym action space
n_actions = env.action_space.shape[0]
n_observations = sum(env.observation_space.shape)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)


steps_done = 0

In [ ]:
num_episodes = 1000
for i_episode in range(num_episodes):
    # Initialize the environment and state
    obs = env.reset()
    obs = torch.from_numpy(obs).to(device).float().unsqueeze(0)
    for t in count():
        # Select and perform an action
        action = select_action(obs, env)
        new_obs, reward, done, metadata = env.step(np.array(action.cpu()))
        new_obs = torch.from_numpy(new_obs).to(device).float().unsqueeze(0)
        reward = torch.tensor([reward], device=device)

        # Store the transition in memory
        memory.push(obs, torch.Tensor(np.array([metadata["action"]], dtype=np.int64)), new_obs, reward)

        # Move to the next state
        obs = new_obs

        # Perform one step of the optimization (on the policy network)
        loss = optimize_model()
        if done:
            print(f"episode #{i_episode}, episode reward: {reward[0]}, episode length: {t+1}, loss: {loss}")
            print(env.state)
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')